In [1]:
import pandas as pd
import datatable as dt
import dask.dataframe as dd
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras 
import keras.backend as K
from tensorflow.keras.callbacks import  EarlyStopping
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.layers import Flatten, concatenate
from tensorflow.keras.layers import BatchNormalization, Dropout, Reshape
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import math
import re
import cmath

In [2]:
df = dt.fread(r"\\wsl.localhost\Ubuntu-22.04\home\imperious\ML and CVAE projects\GW from EMRI\New_GW_signal\Newdatasignal_one_1.txt",sep="\t")
df

,C0,C1,C2,C3,C4,C5,C6,C7,C8,C9,…,C1577903,C1577904,C1577905,C1577906,C1577907
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,3.83985e-22,7.69554e-22,1.13743e-21,1.44821e-21,1.6863e-21,1.83974e-21,1.90084e-21,1.86652e-21,1.73852e-21,1.52325e-21,…,4.45936e-22,−2.50303e-21,−3.98991e-21,−4.32521e-21,−4.13826e-21
1,−1.01416e-21,−7.74709e-22,−4.99028e-22,−2.07651e-22,9.02605e-23,3.85339e-22,6.68307e-22,9.30265e-22,1.16298e-21,1.35912e-21,…,−4.13849e-21,−3.49586e-21,−2.28605e-21,−2.24409e-21,−3.91962e-21
2,1.20966e-21,1.13838e-21,1.04019e-21,9.20266e-22,7.81108e-22,6.25626e-22,4.5707e-22,2.7896e-22,9.50192e-23,−9.09095e-23,…,5.23381e-23,−3.91688e-22,−8.04927e-22,−1.16927e-21,−1.47168e-21
3,−2.42755e-21,−2.25382e-21,−2.06635e-21,−1.87457e-21,−1.67887e-21,−1.47966e-21,−1.27737e-21,−1.0724e-21,−8.65188e-22,−6.5617e-22,…,−1.21095e-21,−1.4834e-21,−1.75128e-21,−2.01394e-21,−2.27078e-21
4,−6.66703e-22,−6.96611e-22,−7.26391e-22,−7.5435e-22,−7.80419e-22,−8.04532e-22,−8.26629e-22,−8.46655e-22,−8.6456e-22,−8.80298e-22,…,−1.01421e-21,−1.02737e-21,−1.03796e-21,−1.04596e-21,−1.05137e-21
5,2.85836e-22,1.08388e-22,−1.08196e-22,−2.9329e-22,−3.93027e-22,−3.78379e-22,−2.53611e-22,−5.50333e-23,1.5956e-22,3.27715e-22,…,−6.98691e-22,1.11654e-21,1.07112e-21,9.94455e-22,1.08191e-21
6,−1.23525e-21,−1.05868e-21,−8.7101e-22,−6.81826e-22,−4.91454e-22,−3.00223e-22,−1.08468e-22,8.34783e-23,2.75282e-22,4.66609e-22,…,−5.17145e-21,−5.07792e-21,−4.97446e-21,−4.86139e-21,−4.73902e-21
7,2.66315e-21,2.78778e-21,2.91335e-21,3.0329e-21,3.14618e-21,3.25295e-21,3.35299e-21,3.4461e-21,3.53209e-21,3.61077e-21,…,8.35826e-23,2.85423e-22,4.86438e-22,6.86225e-22,8.84389e-22
8,8.56012e-21,8.36924e-21,8.14821e-21,7.90709e-21,7.64647e-21,7.36699e-21,7.06935e-21,6.75429e-21,6.42256e-21,6.07501e-21,…,7.88476e-21,8.36311e-21,8.8129e-21,9.2331e-21,9.6228e-21


In [3]:
#for j in range(df.ncols):
#    for i in range(df.nrows):
#        val = df[i, j]
#        if 'I' in val:
#            print(i,j)
#            print(df[i,j])
#            complex_val = complex(val.replace('I', 'j'))
#            print(complex_val)
#            break
#            df[i, j] = str(abs(complex_val))
#        else:
#            df[i, j] = str(val)



In [4]:
params = dt.fread(r"\\wsl.localhost\Ubuntu-22.04\home\imperious\ML and CVAE projects\GW from EMRI\New_GW_signal\paramlist1.txt",sep="\t")
params=params[:250,:]
params = params.to_pandas()

def calcular_formula(formula):
  formula = formula.replace("Pi", str(math.pi))
  return eval(formula)

for i in range(5,9):
  params.iloc[:,i:i+1] = params.iloc[:,i:i+1].map(lambda x: calcular_formula(str(x)))

params

,C0,C1,C2,C3,C4,C5,C6,C7,C8
0,700000,40,0.000,13.874275,0.000000,1.790708,0.596903,0.125664,3.141593
1,800000,72,0.000,14.841555,1.944446,1.162389,0.502655,1.319469,2.356194
2,2000000,34,0.000,9.235850,0.003569,1.53938,2.70177,1.162389,0.973894
3,9000000,86,0.000,7.286453,2.040121,2.356194,1.507964,2.261947,3.110177
4,9000000,34,0.000,6.832572,-0.747715,2.293363,1.476549,1.193805,2.953097
...,...,...,...,...,...,...,...,...,...
238,4000000,66,0.424,7.222384,2.995444,2.890265,2.953097,0.973894,2.764602
239,2000000,89,0.424,9.870412,0.110881,1.53938,0.031416,0.345575,0.219911
240,900000,89,0.424,14.051228,2.472069,2.513274,2.70177,1.507964,2.764602
241,4000000,65,0.424,7.204781,2.342268,2.136283,2.54469,2.858849,2.764602


In [14]:
normalizer_X = MinMaxScaler()
normalizer_X.fit(params) #Fitted with continuous_x. Use it to decode MinMax.

norm_params = normalizer_X.transform(params)

#print(pd.DataFrame(norm_Params))

#normalizing gw data
normalizer_cond = MinMaxScaler()
normalizer_cond.fit(df)

df = normalizer_cond.transform(df) #does sending to the df itself save ram?


In [15]:
hidden_dim = 2
batch_size = 32  #Why does 32 work?

In [16]:
def dropout_and_batchnorm(x):
    return Dropout(0.3)(BatchNormalization()(x))

def noiser(args):
    global mean, log_var
    mean, log_var = args
    N = K.random_normal(shape=(K.shape(mean)[0],hidden_dim), mean=0., stddev=1.0)
    return K.exp(log_var / 2) * N + mean

In [17]:
# Encoder
input_params = Input(shape=(9,))
cond_gw = Input(shape=(1577908,))

x = concatenate([input_params, cond_gw])
x = Dense(64, activation="relu")(x)
x = dropout_and_batchnorm(x)
x = Dense(32, activation="relu")(x)
x = dropout_and_batchnorm(x)

In [18]:
# Latent Space
mean = Dense(hidden_dim)(x)
log_var = Dense(hidden_dim)(x)
h = Lambda(noiser, output_shape=(hidden_dim), name="latent_space")([mean, log_var]) # MAIN ERROR. FIX THE SHAPE.

In [19]:
# Decoder
input_decoder = Input(shape=(hidden_dim,))
cond_gw_decoder = Input(shape=(1577908,))
d = concatenate([input_decoder,cond_gw_decoder])
d = Dense(32, activation="relu")(d)
d = dropout_and_batchnorm(d)
d = Dense(64, activation="relu")(d)
d = dropout_and_batchnorm(d)
decoded = Dense(9, activation="linear")(d)



In [20]:
def vae_loss(x, y):
    loss = K.sum(K.square(x-y), axis=-1)
    kl_loss = -0.5 * K.sum(1 + log_var - K.square(mean) - K.exp(log_var), axis=-1)
    return loss + kl_loss

def relative_precision(true,pred):
    rp = K.sum(K.square(pred - true)/K.square(true))
    return rp
    #sqrt((weighted) sum (estimation - true value)^2/true value^2)


In [21]:
encoder = keras.Model([input_params,cond_gw], h, name="encoder") # Parece estar certo
decoder = keras.Model([input_decoder, cond_gw_decoder], decoded, name="decoder") 

cvae = keras.Model(inputs = [input_params, cond_gw, cond_gw_decoder], #its ok
                   outputs = decoder([encoder([input_params, cond_gw]), cond_gw_decoder]),
                   name="cvae")
optimizer = tf.keras.optimizers.Adam(clipnorm=1.0)

cvae.compile(optimizer="adam", loss=vae_loss,metrics=['mean_absolute_error'])

plot_model(encoder, to_file='encoder_plot.png', show_shapes=True, show_layer_names=True)
plot_model(decoder, to_file='decoder_plot.png', show_shapes=True, show_layer_names=True)

cvae.summary()

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
Model: "cvae"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 9)]          0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 1577908)]    0           []                               
                                                                                                  
 encoder (Functional)           (None, 2)            100989348   ['input_5[0][0]',            

In [22]:
epochs = 25

es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)

cvae.fit(
    [norm_params, df, df], norm_params,
    epochs=epochs,
    shuffle=True,
    validation_split=0.2,
) 

# Why is the loss NaN????   x.x

Epoch 1/25
7/7 [==============================] - 15s 2s/step - loss: 11.3828 - mean_absolute_error: 0.7780 - val_loss: 2.3863 - val_mean_absolute_error: 0.4208
Epoch 2/25
7/7 [==============================] - 9s 1s/step - loss: 7.4979 - mean_absolute_error: 0.5536 - val_loss: 2.2081 - val_mean_absolute_error: 0.3988
Epoch 3/25
7/7 [==============================] - 9s 1s/step - loss: 5.3323 - mean_absolute_error: 0.4505 - val_loss: 2.0473 - val_mean_absolute_error: 0.3803
Epoch 4/25
7/7 [==============================] - 10s 2s/step - loss: 5.0305 - mean_absolute_error: 0.4028 - val_loss: 1.9008 - val_mean_absolute_error: 0.3641
Epoch 5/25
7/7 [==============================] - 10s 1s/step - loss: 5.6658 - mean_absolute_error: 0.4051 - val_loss: 1.7690 - val_mean_absolute_error: 0.3494
Epoch 6/25
7/7 [==============================] - 10s 1s/step - loss: 3.7853 - mean_absolute_error: 0.3609 - val_loss: 1.6480 - val_mean_absolute_error: 0.3371
Epoch 7/25
7/7 [=========================